![ADSA Logo](http://i.imgur.com/BV0CdHZ.png?2 "ADSA Logo")

# Fall 2018 ADSA Workshop - K Nearest Neighbors From Scratch


## [k-Nearest Neighbors][knn] (K-NN)

Fundamentally, this algorithm is
remarkable simple and is based on the principle that data values in an
$N$- dimensional space are generally located near other similar objects.
The number of nearest neighbors, k, is a tuning parameter, and can be
specified a priori or in some algorithms empirically determined. The
basic principle behind k-nn is demonstrated in the following figure from
Wikipedia:

![knn Image from Wikipedia][knni]

As shown in the image, when a new datum is added, the classification
must be assigned. In the case of k-nn, this is done by looking at the
nearest neighbors and using some statistical evaluation of their
classes. For example, we could use some weighted combination of the
nearest neighbors, where the weight might be determined by the relative
distance of each neighbor from the datum of interest. 

In the following code cells, we demonstrate how to perform knn
classification by using scikit-learn. In this example, we use five
nearest neighbors (but this value can be easily adjusted to see how the
classification performance changes). The standard classification
process in scikit-learn is to first fit a model to the training data
and to subsequently apply this model to predict values for the testing
data. After this process, we first compute the prediction score before
displaying the confusion matrix for this algorithm.

-----

[knn]: https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm
[knni]: https://upload.wikimedia.org/wikipedia/commons/thumb/e/e7/KnnClassification.svg/500px-KnnClassification.svg.png

##This workshop will be broken down into six steps
1. Handle Data: Open the dataset from CSV and split into test/train datasets.
2. Similarity: Calculate the distance between two data instances.
3. Neighbors: Locate k most similar data instances.
4. Response: Generate a response from a set of data instances.
5. Accuracy: Summarize the accuracy of predictions.
6. Main: Tie it all together.

##1. Handle Data

In [0]:
from sklearn.datasets import load_iris

iris_dataset = load_iris()

In [2]:
print("Keys of iris_dataset: {}".format(iris_dataset.keys()))

Keys of iris_dataset: dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names'])


In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    iris_dataset['data'], iris_dataset['target'], random_state=0, test_size = 0.2)

In [4]:
print('X_train: ')
print(X_train[0:5])
print(X_train.shape)
print()
print('y_train: ')
print(y_train[0:5])
print(y_train.shape)
print()
print('X_test: ')
print(X_test[0:5])
print(X_test.shape)
print()
print('y_test: ')
print(y_test[0:5])
print(y_test.shape)

X_train: 
[[6.4 3.1 5.5 1.8]
 [5.4 3.  4.5 1.5]
 [5.2 3.5 1.5 0.2]
 [6.1 3.  4.9 1.8]
 [6.4 2.8 5.6 2.2]]
(120, 4)

y_train: 
[2 1 0 2 2]
(120,)

X_test: 
[[5.8 2.8 5.1 2.4]
 [6.  2.2 4.  1. ]
 [5.5 4.2 1.4 0.2]
 [7.3 2.9 6.3 1.8]
 [5.  3.4 1.5 0.2]]
(30, 4)

y_test: 
[2 1 0 2 0]
(30,)


##2. Similarity

For this workshop, we will use the [Euclidean Distance] to calculate the distance between to data instances. This function will allow us to measure the distance between a training data point and a new data point(test data). 

[Euclidean Distance]: https://en.wikipedia.org/wiki/Euclidean_distance

In [0]:
import math

# input: Two arrays of the same length containing measurements. 
# output: The distance between the two arrays

def euclidean_distance(train_data, new_data): 
  # check that two arrays are of the same length
  if len(train_data) != len(new_data): 
    print('ERROR:Data Length Mismatch')
    return()
   
  distance = 0
  for i in range(0,len(train_data)):
    distance += (train_data[i] - new_data[i]) ** 2
    
  return math.sqrt(distance)

##3. Neighbors
Now that we have a function that calculates the distance between two points, we can get calculate distance between our training data and new data point. We then want to find the k minimum distances and the label of the corresponding training data. 

In [0]:
import operator

#input: 
# train_data - a 2d-array containing the training data
# train_labels - the labels for the training data
# new_data - the data that is to be classfied 
# k - the number of neighbors to return

# output: a list of k labels of the nearest neighbors

def nearest_neighbors(train_data, train_labels, new_data, k):
  distances_list = []
  
  # iterate through the training data
  for i in range(0, len(train_data)):
    # get the distance between one point of the training data and the new data
    distance = euclidean_distance(train_data[i], new_data)
    
    # append a tuple containing the label and the distance to the list
    distances_list.append((train_labels[i],distance))
    
  # sort the list by the distance by ascending order
  distances_list = sorted(distances_list,key=operator.itemgetter(1))
  
  # return the first k elements of sorted list
  return distances_list[:k]

In [7]:
data = [[1,2,3,4],[2,3,4,5]]
labels = [0,1]
new_data = [1,1,1,1]
k = 1

neighbors = nearest_neighbors(data,labels,new_data,k)
print(neighbors)

[(0, 3.7416573867739413)]


## 4. Response
Now that we have the k minimum distances and their corresponding label, we can formulate a response for each point of our test data by finding the label that appears most often. 

In [0]:
predicted_labels = []
k = 1

# iterate through the test data
for i in range(0, len(X_test)):
  # get the k nearest neighbors for the current test data array
  neighbors = nearest_neighbors(X_train, y_train, X_test[i], k)
  
  # dictionary that keeps track of how many times a label appears
  response_dictionary = {}
  
  # iterate through the k nearest neighbors
  for neighbor in neighbors: 
    label = neighbor[0]
    
    # if the label exists in the dictionary then increment
    if label in response_dictionary: 
      response_dictionary[label] += 1
      
    # otherwise insert label into dictionary
    else: 
      response_dictionary.update({label:1})
  
  max_label = None
  max_count = -1
  
  # find label that appeared most often by iterating through the dictionary
  for key,value in response_dictionary.items(): 
    
    # check if there is a tie
    if value == max_count and max_label != key: 
      print('max_count: ' + str(max_count))
      print('max_label: ' + str(max_label))
      print('value: ' + str(value))
      print('key: ' + str(key))
      print('Tie: Change value of k')
      break
      
    if value > max_count: 
      max_count = value
      max_label = key
  
  
  predicted = max_label
  
  predicted_labels.append(predicted)

In [9]:
print(predicted_labels)

[2, 1, 0, 2, 0, 2, 0, 1, 1, 1, 2, 1, 1, 1, 1, 0, 1, 1, 0, 0, 2, 1, 0, 0, 2, 0, 0, 1, 1, 0]


##5. Accuracy
Now that we have our predicted labels, we can check how many of them were correct and get a accuract percentage. 

In [0]:
correct_count = 0
total_count   = 0

for i in range(0, len(y_test)): 
  if y_test[i] == predicted_labels[i]:
    correct_count += 1
  total_count += 1
accuracy = correct_count/total_count

In [11]:
print(accuracy)

1.0


##6. Main
Now that we have all of our pieces, we can put it all together into a class

In [0]:
def euclidean_distance(train_data, new_data): 
  # check that two arrays are of the same length
  if len(train_data) != len(new_data): 
    print('ERROR:Data Length Mismatch')
    return()

  distance = 0
  for i in range(0,len(train_data)):
    distance += (train_data[i] - new_data[i]) ** 2

  return math.sqrt(distance)

def nearest_neighbors(train_data, train_labels, new_data, k):
  distances_list = []
  
  for i in range(0, len(train_data)):
    distance = euclidean_distance(train_data[i], new_data)
    
    distances_list.append((train_labels[i],distance))
    
  distances_list = sorted(distances_list,key=operator.itemgetter(1))
  
  return distances_list[:k]

class KNN: 
  def __init__(self,k): 
    self.train_data = None
    self.train_labels = None
    self.k_value = k
    
  def fit(self,train,labels): 
    self.train_data = train
    self.train_labels = labels
    
  def predict(self,test_data): 
    predicted_labels = []
    
    for i in range(0, len(test_data)):
      neighbors = nearest_neighbors(self.train_data, self.train_labels, test_data[i], self.k_value)

      response_dictionary = {}

      for neighbor in neighbors: 
        label = neighbor[0]

        if label in response_dictionary: 
          response_dictionary[label] += 1

        else: 
          response_dictionary.update({label:1})

      max_label = None
      max_count = -1

      for key,value in response_dictionary.items(): 

        if value == max_count: 
          print('max_count: ' + str(max_count))
          print('max_label: ' + str(max_label))
          print('value: ' + str(value))
          print('key: ' + str(key))
          print('Tie: Change value of k')
          return

        if value > max_count: 
          max_count = value
          max_label = key


      predicted = max_label

      predicted_labels.append(predicted)
      
    return predicted_labels
  
  def score(self,test_data, test_labels): 
    predict = self.predict(test_data)
    
    correct_count = 0
    total_count   = 0

    for i in range(0, len(test_labels)): 
      if test_labels[i] == predict[i]:
        correct_count += 1
      total_count += 1
      
    accuracy = correct_count/total_count
    
    return accuracy

In [0]:
X_train, X_test, y_train, y_test = train_test_split(
    iris_dataset['data'], iris_dataset['target'], random_state=0, test_size = 0.2)


In [0]:
knn = KNN(k=3)

In [0]:
knn.fit(X_train,y_train)

In [16]:
predict = knn.predict(X_test)
print(predict)

[2, 1, 0, 2, 0, 2, 0, 1, 1, 1, 2, 1, 1, 1, 2, 0, 1, 1, 0, 0, 2, 1, 0, 0, 2, 0, 0, 1, 1, 0]


In [17]:
score = knn.score(X_test,y_test)
print(score)

0.9666666666666667


##8. Using sklearn's implementation

In [18]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

print(knn.score(X_test, y_test))

0.9666666666666667
